In [1]:
import tensorflow as tf

mnist = tf.keras.datasets.mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

2023-05-30 18:09:49.492974: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-30 18:09:49.984653: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-30 18:09:49.984688: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-05-30 18:09:49.984692: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
from keras.layers import Dense, Concatenate, Reshape, Input, Conv2D, ReLU, BatchNormalization, GlobalAveragePooling2D
from keras.models import Sequential, Model

img_A_inp = Input((28,28), name='inpA')
img_B_inp = Input((28,28), name='inpB')

def get_cnn_block(depth):
    return Sequential([Conv2D(depth, 3, 1),
                        BatchNormalization(),
                       ReLU()])
DEPTH = 64
cnn = Sequential([Reshape((28, 28,1)),
                  get_cnn_block(DEPTH),
                  get_cnn_block(DEPTH*2),
                  get_cnn_block(DEPTH*4),
                  get_cnn_block(DEPTH*8),
                  GlobalAveragePooling2D(),
                  Dense(64, activation='relu')])

feature_vector_A = cnn(img_A_inp)
feature_vector_B = cnn(img_B_inp)

concat = Concatenate()([feature_vector_A, feature_vector_B])
dense = Dense(64, activation='relu')(concat)
output = Dense(1, activation='sigmoid')(dense)

model = Model(inputs=[img_A_inp, img_B_inp], outputs=output)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inpA (InputLayer)              [(None, 28, 28)]     0           []                               
                                                                                                  
 inpB (InputLayer)              [(None, 28, 28)]     0           []                               
                                                                                                  
 sequential_4 (Sequential)      (None, 64)           1586496     ['inpA[0][0]',                   
                                                                  'inpB[0][0]']                   
                                                                                                  
 concatenate (Concatenate)      (None, 128)          0           ['sequential_4[0][0]',       

2023-05-30 18:09:50.549516: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-30 18:09:50.552770: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-30 18:09:50.552890: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-30 18:09:50.553261: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

In [3]:
import numpy as np
random_indices = np.random.choice(X_train.shape[0], 300, replace=False)
X_train_sample, y_train_sample = X_train[random_indices], y_train[random_indices]
X_train_sample.shape, y_train_sample.shape

((300, 28, 28), (300,))

In [4]:
import itertools
def make_paired_dataset(X, y):
    X_pairs, y_labels = [],[]
    tuples = [(x1,y1) for x1, y1 in zip(X,y)]
    for t in itertools.product(tuples, tuples):
        pair_A, pair_B = t
        img_A, label_A =  t[0]
        img_B, label_B =  t[1]

        new_label = int(label_A==label_B)
        X_pairs.append([img_A, img_B])
        y_labels.append(new_label)
    X_pairs = np.array(X_pairs)
    y_labels = np.array(y_labels)
    return X_pairs, y_labels

In [5]:
X_train_pairs, y_train_labels = make_paired_dataset(X_train_sample, y_train_sample)

In [6]:
X_train_pairs.shape, y_train_labels.shape

((90000, 2, 28, 28), (90000,))

In [7]:
random_indices = np.random.choice(X_test.shape[0], 150, replace=False)
X_test_sample, y_test_sample = X_test[random_indices], y_test[random_indices]
X_test_sample.shape, y_test_sample.shape

((150, 28, 28), (150,))

In [8]:
X_test_pairs, y_test_labels = make_paired_dataset(X_test_sample, y_test_sample)

In [9]:
X_test_pairs.shape, y_test_labels.shape

((22500, 2, 28, 28), (22500,))

In [10]:
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])

In [11]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(patience=3)

In [12]:

model.fit(x=[X_train_pairs[:,0,:,:], X_train_pairs[:,1,:,:]],
          y = y_train_labels,
          validation_data = ([X_test_pairs[:,0,:,:], X_test_pairs[:,1,:,:]],
                             y_test_labels
                             ),
          epochs=1000,
          batch_size=32,
          callbacks=[es]
          )

Epoch 1/1000


2023-05-30 18:09:52.649002: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8101
2023-05-30 18:09:52.760419: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-05-30 18:09:53.024677: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f6148017dc0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-30 18:09:53.024699: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce GTX 1080, Compute Capability 6.1
2023-05-30 18:09:53.033690: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-05-30 18:09:53.115608: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-05-30 18:09:53.146101: I tensorflow/compiler/jit/xla_compilati

2813/2813 [==============================] - 121s 42ms/step - loss: 0.1936 - accuracy: 0.9252 - val_loss: 0.1699 - val_accuracy: 0.9347
Epoch 2/1000
2813/2813 [==============================] - 115s 41ms/step - loss: 0.0370 - accuracy: 0.9871 - val_loss: 0.1746 - val_accuracy: 0.9447
Epoch 3/1000
2813/2813 [==============================] - 110s 39ms/step - loss: 0.0118 - accuracy: 0.9964 - val_loss: 0.1359 - val_accuracy: 0.9561
Epoch 4/1000
2813/2813 [==============================] - 111s 40ms/step - loss: 0.0050 - accuracy: 0.9987 - val_loss: 0.1401 - val_accuracy: 0.9680
Epoch 5/1000
2813/2813 [==============================] - 105s 37ms/step - loss: 0.0055 - accuracy: 0.9983 - val_loss: 0.1463 - val_accuracy: 0.9673
Epoch 6/1000
2813/2813 [==============================] - 114s 41ms/step - loss: 0.0020 - accuracy: 0.9994 - val_loss: 0.1437 - val_accuracy: 0.9686


In [32]:
y_test[1], y_test[44]

(2, 3)

In [33]:
model.predict([X_test[1].reshape((1,28,28)), X_test[44].reshape((1,28,28))])

1/1 [==============================] - 0s 15ms/step


array([[2.2042728e-07]], dtype=float32)

In [34]:
model.save('test_network_mnist')

INFO:tensorflow:Assets written to: test_network_mnist/assets


INFO:tensorflow:Assets written to: test_network_mnist/assets
